# 0 - Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

#ROOT_PATH = "/content/drive/MyDrive/EPFL/optMLProject_Nicolas²_Oliver/" # Nicolas P.
ROOT_PATH = "/content/drive/MyDrive/optMLProject_Nicolas²_Oliver/" # Nicolas B.
#ROOT_PATH = "/content/drive/MyDrive/ ... / optMLProject_Nicolas²_Oliver/" # Oliver D.

print(os.listdir(ROOT_PATH))

Mounted at /content/drive
['.DS_Store', 'README.md', '.vscode', '.git', 'models', 'utils', 'data', 'optimizers', 'Hyperparameters Search.gdoc', 'more_info_hyperparameters_search.xlsx', 'optML.ipynb', 'hyperparameters_search.xlsx']


In [2]:
%load_ext autoreload
%autoreload 2

#Imports

import sys
sys.path.append(ROOT_PATH)

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

!pip install Lion_pytorch
from lion_pytorch import Lion

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


from utils import constants as cst
from utils import auxiliaries as aux

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1 - Grid-Search for Hyperparameters  

In [3]:
########### 1st cell ###########

######### TO MANUALLY SET UP #########

# choices : resnet18, resnet101
model_name = "resnet18" 

# choices : SGD, SGDM, RMS, ADAM, LION
optimizer_name = "LION"

######### TO MANUALLY SET UP #########

# Just checking
assert model_name in ["resnet18", "resnet101"]
assert optimizer_name in ["SGD", "SGDM", "RMS", "ADAM", "LION"]

In [4]:
########### 2nd cell ###########

# Retrieve the hyperparamaters 
hyperparams_sheet_path = ROOT_PATH + "hyperparameters_search.xlsx"
hyperparams = aux.get_hyperparams(model_name, optimizer_name, hyperparams_sheet_path)


print(f"HYPERPARAMS CHECK : {hyperparams}")

### The path for saving the best model ###
model_path = ROOT_PATH + "models/"  + model_name + "/"
assert os.path. isdir(model_path)

file_name = model_name + '-' + optimizer_name
save_path = model_path + file_name
print(f"SAVE-PATH CHECK : {save_path}")
### The path for saving the best model ###

# Creates the config
config = dict(
    model=model_name,
    optimizer=optimizer_name,
)
print(f"CONFIG CHECK : {config}")

# Creates the folds 
folds = aux.get_folds()

HYPERPARAMS CHECK : {'lr': [9e-06, 3e-05], 'weight_decay': [1.0, 0.5], 'momentum': [0.0]}
SAVE-PATH CHECK : /content/drive/MyDrive/optMLProject_Nicolas²_Oliver/models/resnet18/resnet18-LION
CONFIG CHECK : {'model': 'resnet18', 'optimizer': 'LION'}


100%|██████████| 169001437/169001437 [00:05<00:00, 28247362.21it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
########### 3rd cell ###########

######### RUN the Grid-Search #########
(all_accuracies, 
all_hyperparams, 
best_accuracy, 
best_hyperparams) = aux.grid_search(
    model_name, 
    optimizer_name, 
    hyperparams, 
    folds,
    save_path)

# Store the accuracies for all hyperparameters combinations in the sheet
aux.store_grid_search_results(all_accuracies, all_hyperparams, best_accuracy.item(), best_hyperparams, model_name, optimizer_name, hyperparams_sheet_path)

--- Working with LION on resnet18 ---
Hyperparameters: 
Learning Rate : 9e-06 
Weight Decay : 1.0 
Momentum : 0.0
----- Training on fold # 1 -----
Epoch: 1|10


Training: 100%|██████████| 157/157 [00:23<00:00,  6.76it/s]


Epoch: 2|10


Training: 100%|██████████| 157/157 [00:22<00:00,  6.86it/s]


Epoch: 3|10


...

Epoch: 10|10


Training: 100%|██████████| 157/157 [00:21<00:00,  7.14it/s]


Average lost on the last bacthes : -39.99157028198242


Testing: 100%|██████████| 10/10 [00:05<00:00,  1.96it/s]


Accuracy : 0.15649999678134918

 Best Hyperparameters so far, Model Saved !
Best Accuracy : 0.15649999678134918
